In [1]:
import os, sys
import gc
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('../')
import torch

In [3]:
from get_algos import get_all_algos , run_experiment ,create_paths

In [4]:
import gym
from gym.wrappers import Monitor
from stable_baselines3 import PPO, A2C, DDPG, DQN, SAC, TD3
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env

### 1. Explore breakout environment
<ul>
    <li>Explore the environment</li>
</ul>

In [5]:
env_name='Breakout-v0'
env=gym.make(env_name)

In [6]:
print('Action space: ',env.action_space)
print('Observation space shape: ',env.observation_space)
print('Action space shape: ',env.action_space.shape)
print('Observation space shape: ',env.observation_space.shape)

Action space:  Discrete(4)
Observation space shape:  Box([[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [

#Taking a look at how breakout looks and simulate random actions
env.reset()
rewards_all=[]
for epi in range(50):
    state=env.reset()
    rewards=0
    while True:
        action=env.action_space.sample()
        state,reward,done,_=env.step(action)
        env.render()
        rewards+=reward
        if done:
            rewards_all.append(rewards)
            if (epi+1)%5==0:
                print('Episode: ',epi+1,' Reward: ',rewards)
            break  
env.close()
print('Average rewards for random action Breakout-v0 is: ',sum(rewards_all)/len(rewards_all))

### 2. Training
<ol>
    <li>Make the atari env using make_atari_env</li>
    <li>Parallelize the learning using VecFrameStack</li>
    <li>Create train function for all appropriate algos</li>
    <li>Train</li>
    <li>Store model</li>
</ol> 

In [17]:
def train_breakout(algo_name, env_name,n_steps,device='cuda'):
    if algo_name=='DQN': #GPU was unable to load the entire experience relay so moved to virtual memory on disk
        device='cpu'
    log_path, render_path, model_path=create_paths(algo_name,env_name,n_steps)
    env=make_atari_env(env_name)
    env=VecFrameStack(env, n_stack=6)
    model=eval(algo_name)('CnnPolicy',env, tensorboard_log=log_path,verbose=1,device=device)
    model.learn(n_steps)
    model.save(os.path.join(model_path,env_name+"_"+algo_name+"_"+"model"))
    del model
    env.close()

    return log_path, render_path, model_path

In [9]:
algo_list=get_all_algos(gym.make(env_name))
print(algo_list)

['PPO', 'A2C', 'DQN']


In [10]:
models_records={}
n_steps=200000

In [11]:
def train_experiment(algo_list, env_name,n_steps, models_records):
    for algo in reversed(algo_list):
        print('******* Training ',algo,' *******')
        model_record={}
        log_path, render_path, model_path=train_breakout(algo, env_name,n_steps)
        model_record['log']=log_path
        model_record['render']=render_path
        model_record['model']=model_path
        models_records[algo]=model_record
        gc.collect()
        torch.cuda.empty_cache()

In [12]:
train_experiment(algo_list, env_name, 200000, models_records)

******* Training  DQN  *******
Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to Breakout-v0\DQN_200000\logs\DQN_1


E:\Anaconda\envs\ai_gym\lib\site-packages\stable_baselines3\common\buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 84.69GB > 13.06GB
  "This system does not have apparently enough memory to store the complete "


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 206      |
|    time_elapsed     | 0        |
|    total_timesteps  | 40       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 262      |
|    ep_rew_mean      | 1        |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 264      |
|    time_elapsed     | 0        |
|    total_timesteps  | 83       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 256      |
|    ep_rew_mean      | 1        |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 315      |
|    time_elapsed   

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 298      |
|    ep_rew_mean      | 1.72     |
|    exploration_rate | 0.95     |
| time/               |          |
|    episodes         | 92       |
|    fps              | 351      |
|    time_elapsed     | 2        |
|    total_timesteps  | 1046     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 295      |
|    ep_rew_mean      | 1.68     |
|    exploration_rate | 0.949    |
| time/               |          |
|    episodes         | 96       |
|    fps              | 349      |
|    time_elapsed     | 3        |
|    total_timesteps  | 1072     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 294      |
|    ep_rew_mean      | 1.65     |
|    exploration_rate | 0.947    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 279      |
|    ep_rew_mean      | 1.44     |
|    exploration_rate | 0.911    |
| time/               |          |
|    episodes         | 180      |
|    fps              | 316      |
|    time_elapsed     | 5        |
|    total_timesteps  | 1867     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 279      |
|    ep_rew_mean      | 1.44     |
|    exploration_rate | 0.909    |
| time/               |          |
|    episodes         | 184      |
|    fps              | 318      |
|    time_elapsed     | 6        |
|    total_timesteps  | 1924     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 282      |
|    ep_rew_mean      | 1.49     |
|    exploration_rate | 0.907    |
| time/               |          |
|    episodes       

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 531        |
|    ep_rew_mean          | 6.85       |
| time/                   |            |
|    fps                  | 143        |
|    iterations           | 55         |
|    time_elapsed         | 783        |
|    total_timesteps      | 112640     |
| train/                  |            |
|    approx_kl            | 0.19280937 |
|    clip_fraction        | 0.408      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.515     |
|    explained_variance   | 0.767      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.102     |
|    n_updates            | 540        |
|    policy_gradient_loss | -0.0745    |
|    value_loss           | 0.0584     |
----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 543       |
|    ep_rew_mean   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 538        |
|    ep_rew_mean          | 7.18       |
| time/                   |            |
|    fps                  | 143        |
|    iterations           | 65         |
|    time_elapsed         | 926        |
|    total_timesteps      | 133120     |
| train/                  |            |
|    approx_kl            | 0.22427694 |
|    clip_fraction        | 0.383      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.441     |
|    explained_variance   | 0.821      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.087     |
|    n_updates            | 640        |
|    policy_gradient_loss | -0.069     |
|    value_loss           | 0.0523     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 542        |
|    ep_rew_mean

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 556       |
|    ep_rew_mean          | 7.49      |
| time/                   |           |
|    fps                  | 143       |
|    iterations           | 75        |
|    time_elapsed         | 1069      |
|    total_timesteps      | 153600    |
| train/                  |           |
|    approx_kl            | 0.2784238 |
|    clip_fraction        | 0.384     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.419    |
|    explained_variance   | 0.679     |
|    learning_rate        | 0.0003    |
|    loss                 | -0.0837   |
|    n_updates            | 740       |
|    policy_gradient_loss | -0.0669   |
|    value_loss           | 0.0708    |
---------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 565        |
|    ep_rew_mean          | 7.7      

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 531        |
|    ep_rew_mean          | 6.95       |
| time/                   |            |
|    fps                  | 143        |
|    iterations           | 85         |
|    time_elapsed         | 1209       |
|    total_timesteps      | 174080     |
| train/                  |            |
|    approx_kl            | 0.34060726 |
|    clip_fraction        | 0.384      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.416     |
|    explained_variance   | 0.704      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0891    |
|    n_updates            | 840        |
|    policy_gradient_loss | -0.0677    |
|    value_loss           | 0.0643     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 529        |
|    ep_rew_mean

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 541        |
|    ep_rew_mean          | 7.14       |
| time/                   |            |
|    fps                  | 143        |
|    iterations           | 95         |
|    time_elapsed         | 1351       |
|    total_timesteps      | 194560     |
| train/                  |            |
|    approx_kl            | 0.38957772 |
|    clip_fraction        | 0.395      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.424     |
|    explained_variance   | 0.564      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0748    |
|    n_updates            | 940        |
|    policy_gradient_loss | -0.0693    |
|    value_loss           | 0.054      |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 551        |
|    ep_rew_mean

In [13]:
models_records.keys()

dict_keys(['DQN', 'A2C', 'PPO'])